In [1]:
!pip install -q kaggle

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amitkkumar1","key":"21d3ebc454490d5e03f116e3aa40cf2d"}'}

In [5]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria

mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 675M/675M [00:31<00:00, 22.8MB/s]
100% 675M/675M [00:31<00:00, 22.7MB/s]


In [6]:
import os
import zipfile

In [7]:
os.listdir()

['.config',
 'kaggle (1).json',
 'cell-images-for-detecting-malaria.zip',
 'kaggle.json',
 'sample_data']

In [8]:
file = 'cell-images-for-detecting-malaria.zip'
zip_ref = zipfile.ZipFile(file, 'r')
zip_ref.extractall()
zip_ref.close()

In [9]:
dir_name = './cell_images/cell_images'
os.listdir(dir_name)

['Uninfected', 'Parasitized']

In [10]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [11]:
# ImageDataGenerator class provides the abilities to process image data efficiently
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
tf.random.set_seed(12345)

In [13]:
# Define the Dataset Generator. We will use this to read files. Validation split
# determines the split into training and testing
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   validation_split = 0.2,
                                   horizontal_flip = True)

In [14]:
# Read the training set
training_set = train_datagen.flow_from_directory(dir_name,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 subset='training',
                                                 class_mode = 'binary')

Found 22048 images belonging to 2 classes.


In [15]:
validation_set = train_datagen.flow_from_directory(dir_name,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 subset='validation',
                                                 class_mode = 'binary')

Found 5510 images belonging to 2 classes.


In [16]:
print(dir_name)

./cell_images/cell_images


In [17]:
training_set.class_indices, validation_set.class_indices

({'Parasitized': 0, 'Uninfected': 1}, {'Parasitized': 0, 'Uninfected': 1})

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D,Flatten, Dense

In [19]:
model = Sequential() # Create a blank model

In [20]:
# Add the first Convolution Layer and Max Pool layer
model.add(Conv2D(name='cnv_layer',
                 filters=32,
                 kernel_size=3,
                 activation='relu',
                 padding='same',
                 input_shape=[224, 224, 3]))

In [21]:
model.add(MaxPool2D(pool_size = 2)) # Maxpool Layer

In [22]:
model.add(Flatten(name='flatten')) # Add the flatten layer

In [23]:
model.add(Dense(units = 64,
                name='dense',
                activation = 'relu')) #Add the dense layer

In [24]:
model.add(Dense(name='output',
                units=1,
                activation = 'sigmoid')) # Final Output layer

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnv_layer (Conv2D)          (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 401408)            0         
                                                                 
 dense (Dense)               (None, 64)                25690176  
                                                                 
 output (Dense)              (None, 1)                 65        
                                                                 
Total params: 25,691,137
Trainable params: 25,691,137
Non-trainable params: 0
____________________________________________

In [27]:
(3*3*3+1)*32, 112*112*32, (401408+1)*64

(896, 401408, 25690176)

In [28]:
# Compile the model
model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [25]:
model.fit(training_set,
          validation_data = validation_set,
          epochs=3) # Train the model for 3 epochs

Epoch 1/3
689/689 [==============================] - 322s 451ms/step - loss: 0.7739 - accuracy: 0.6987 - val_loss: 0.5298 - val_accuracy: 0.7991
Epoch 2/3
689/689 [==============================] - 299s 434ms/step - loss: 0.3832 - accuracy: 0.8528 - val_loss: 0.6421 - val_accuracy: 0.8828
Epoch 3/3
689/689 [==============================] - 302s 438ms/step - loss: 0.2760 - accuracy: 0.9043 - val_loss: 0.2814 - val_accuracy: 0.9016


# Transfer Learning

In [29]:
from tensorflow.keras.applications import MobileNetV2

In [30]:
mobile_net = MobileNetV2(input_shape=(224, 224, 3), include_top=False)
mobile_net.trainable = False

9406464/9406464 [==============================] - 1s 0us/step


In [31]:
trf_model = Sequential()

In [32]:
trf_model.add(mobile_net)

In [33]:
trf_model.add(Flatten())
trf_model.add(Dense(256, activation = 'relu'))

In [34]:
trf_model.add(Dense(1, activation = 'sigmoid'))

In [35]:
trf_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 256)               16056576  
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 18,314,817
Trainable params: 16,056,833
Non-trainable params: 2,257,984
_________________________________________________________________


In [36]:
# Compile the model
trf_model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [37]:
trf_model.fit(training_set,
              validation_data = validation_set,
              epochs = 3)

Epoch 1/3
689/689 [==============================] - 343s 483ms/step - loss: 0.5540 - accuracy: 0.9106 - val_loss: 0.2055 - val_accuracy: 0.9230
Epoch 2/3
689/689 [==============================] - 325s 472ms/step - loss: 0.1598 - accuracy: 0.9418 - val_loss: 0.1990 - val_accuracy: 0.9260
Epoch 3/3
689/689 [==============================] - 330s 480ms/step - loss: 0.1494 - accuracy: 0.9456 - val_loss: 0.1791 - val_accuracy: 0.9348
